## Efficient Optimization Algorithms

Optuna는 HyperParameter sampling을 위한 SOTA algorithm을 채택하고 효율적이지 못한 trials를 pruning할 수 있다.  

Optuna는 다음의 Algorithm을 제공한다.

- Tree-structured Parzen Estimator algorithm implemented in :class:`optuna.samplers.TPESampler`

- CMA-ES based algorithm implemented in :class:`optuna.samplers.CmaEsSampler`

- Grid Search implemented in :class:`optuna.samplers.GridSampler`

- Random Search implemented in :class:`optuna.samplers.RandomSampler`

The default sampler is :class:`optuna.samplers.TPESampler`.

## Switching Samplers

In [1]:
import optuna

In [3]:
# Default Sampler는 TPESampler
study = optuna.create_study()
print(f"Sampler is {study.sampler.__class__.__name__}")

[I 2021-11-26 14:59:22,851] A new study created in memory with name: no-name-8d4f3d7c-2387-4ae6-9c66-183c7afe7482


Sampler is TPESampler


In [4]:
# 다른 Sampler를 활용하고 싶을 경우, sampler 옵션 활용
study = optuna.create_study(sampler=optuna.samplers.RandomSampler())
print(f"Sampler is {study.sampler.__class__.__name__}")

study = optuna.create_study(sampler=optuna.samplers.CmaEsSampler())
print(f"Sampler is {study.sampler.__class__.__name__}")

[I 2021-11-26 14:59:52,884] A new study created in memory with name: no-name-80a9c90c-dcfc-427b-ba18-5f79cf6509a5
[I 2021-11-26 14:59:52,888] A new study created in memory with name: no-name-9b5cadff-60d2-4f70-9073-8f3a491bec6d


Sampler is RandomSampler
Sampler is CmaEsSampler


## Pruning Algorithms

``Pruners``가 안좋은 trials에 대해서는 훈련의 앞쪽에서 자동으로 멈추게 만든다. (a.k.a automated early-stopping)

Optuna는 다음의 Prunig Algorithm을 제공한다.

- Asynchronous Successive Halving algorithm implemented in :class:`optuna.pruners.SuccessiveHalvingPruner`

- Hyperband algorithm implemented in :class:`optuna.pruners.HyperbandPruner`

- Median pruning algorithm implemented in :class:`optuna.pruners.MedianPruner`

- Threshold pruning algorithm implemented in :class:`optuna.pruners.ThresholdPruner`

We use :class:`optuna.pruners.MedianPruner` in most examples. 성능 역시 다른 pruning algorithm보다 우수하다.

## Activating Pruners


pruning을 하기 위해서 학습 중에 각 step에서 report와 should_prune을 호출 해야한다.
- ``optuna.trial.Trial.report``: 중간 objective 값을 모니터링한다.
- ``optuna.trial.Trial.should_prune``: 미리 정의된 조건을 충족하지 않으면 trial을 종료한다.

We would recommend using integration modules for major machine learning frameworks. [Github-Optuna](https://github.com/optuna/optuna-examples/)

In [7]:
import logging
import sys

import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection

def objective(trial):
    iris = sklearn.datasets.load_iris() # iris data laod
    classes = list(set(iris.target))

    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(
        iris.data, iris.target, test_size = 0.25, random_state = 0
    )

    alpha = trial.suggest_float('alpha', 1e-5, 1e-1, log=True)
    clf = sklearn.linear_model.SGDClassifier(alpha=alpha)

    for step in range(100) :
        clf.partial_fit(train_x, train_y, classes=classes)
        
        # Report intermediate objective value
        intermediate_value = 1.0 - clf.score(valid_x, valid_y)
        trial.report(intermediate_value, step)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.TrialPruned()

    return 1.0 - clf.score(valid_x, valid_y)

In [9]:
# Add stream handler of stdout to show the messages
# optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

[I 2021-11-26 15:19:18,578] A new study created in memory with name: no-name-146afbe2-c7ea-41c7-9db9-195daddec8dc


A new study created in memory with name: no-name-146afbe2-c7ea-41c7-9db9-195daddec8dc
A new study created in memory with name: no-name-146afbe2-c7ea-41c7-9db9-195daddec8dc


[I 2021-11-26 15:19:18,853] Trial 0 finished with value: 0.052631578947368474 and parameters: {'alpha': 0.00563340118988228}. Best is trial 0 with value: 0.052631578947368474.


Trial 0 finished with value: 0.052631578947368474 and parameters: {'alpha': 0.00563340118988228}. Best is trial 0 with value: 0.052631578947368474.
Trial 0 finished with value: 0.052631578947368474 and parameters: {'alpha': 0.00563340118988228}. Best is trial 0 with value: 0.052631578947368474.


[I 2021-11-26 15:19:19,087] Trial 1 finished with value: 0.368421052631579 and parameters: {'alpha': 0.00014237215428023828}. Best is trial 0 with value: 0.052631578947368474.


...

Trial 17 pruned. 
Trial 17 pruned. 


[I 2021-11-26 15:19:20,430] Trial 18 pruned. 


Trial 18 pruned. 
Trial 18 pruned. 


[I 2021-11-26 15:19:20,682] Trial 19 finished with value: 0.26315789473684215 and parameters: {'alpha': 0.0016820260534773238}. Best is trial 0 with value: 0.052631578947368474.


Trial 19 finished with value: 0.26315789473684215 and parameters: {'alpha': 0.0016820260534773238}. Best is trial 0 with value: 0.052631578947368474.
Trial 19 finished with value: 0.26315789473684215 and parameters: {'alpha': 0.0016820260534773238}. Best is trial 0 with value: 0.052631578947368474.


## Which Sampler and Pruner Should be Used?

- `optuna.samplers.RandomSampler` with `optuna.pruners.MedianPruner` is the best.
- `optuna.samplers.TPESampler` with `optuna.pruners.Hyperband` is the best.

However, note that the benchmark is not deep learning.  

For deep learning tasks,consult the below table.  
This table is from the `Ozaki et al., Hyperparameter Optimization Methods: Overview and Characteristics, in IEICE Trans, Vol.J103-D No.9 pp.615-631, 2020 <https://doi.org/10.14923/transinfj.2019JDR0003>`_ paper,
which is written in Japanese.

+---------------------------+-----------------------------------------+---------------------------------------------------------------+
| Parallel Compute Resource | Categorical/Conditional Hyperparameters | Recommended Algorithms                                        |
+===========================+=========================================+===============================================================+
| Limited                   | No                                      | TPE. GP-EI if search space is low-dimensional and continuous. |
+                           +-----------------------------------------+---------------------------------------------------------------+
|                           | Yes                                     | TPE. GP-EI if search space is low-dimensional and continuous  |
+---------------------------+-----------------------------------------+---------------------------------------------------------------+
| Sufficient                | No                                      | CMA-ES, Random Search                                         |
+                           +-----------------------------------------+---------------------------------------------------------------+
|                           | Yes                                     | Random Search or Genetic Algorithm                            |
+---------------------------+-----------------------------------------+---------------------------------------------------------------+


## Integration Modules for Pruning
Optuna는 ``integraion`` module을 제공하는 데, 이를 활용하여 puning을 간단하게 실행할 수 있다.

다음 처럼 활용할 수 있다.
-> visualization.ipynb에서 확인하자.

```python
pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
bst = xgb.train(param, dtrain, evals=[(dvalid, 'validation')], callbacks=[pruning_callback])
```